In [ ]:
!pip install -q spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [ ]:
cid = '-------'
secret = '------'

In [ ]:
playlists = sp.user_playlists('thelowering')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
#auth
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/6t1fydb2qLBEvCNcmNx9pc?si=6e2c34f4c85447e6"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
len(track_uris)

In [ ]:
import pandas as pd

In [ ]:
def get_playlist_tracks(username,playlist_uri):
    results = sp.user_playlist_tracks(username,playlist_uri)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
my_tracks = get_playlist_tracks('thelowering', '6t1fydb2qLBEvCNcmNx9pc')

In [ ]:
df = pd.DataFrame(columns=['track_uri', 'track_name', 'track_pop', 'artist_uri', 'artist_name', 'artist_pop', 'artist_genres', 'album'])
for track in my_tracks: #sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    if len(artist_info["genres"]) == 0:
        artist_genres = 'none'
    else:
        artist_genres = artist_info["genres"][0]
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    row = {'track_uri':track_uri[14:], 'track_name':track_name, 'track_pop':track_pop, 'artist_uri':artist_uri[15:], 
           'artist_name':artist_name, 'artist_pop':artist_pop, 'artist_genres':artist_genres, 'album':album}
    df = df.append(row, ignore_index=True)

In [ ]:
df

In [ ]:
import pickle
with open('my_songs_df.pickle', 'wb') as my_songs_df:
    pickle.dump(df, my_songs_df, protocol=pickle.HIGHEST_PROTOCOL)